In [10]:
%%time
import torch
import numpy as np
from scipy.optimize import minimize, Bounds
import cProfile, io, pstats
from pstats import SortKey
from botorch.models.gp_regression import SingleTaskGP
from botorch.acquisition import qNoisyExpectedImprovement
from gpytorch import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_mll

torch.manual_seed(0)
np.random.seed(0)

torch.set_default_dtype(torch.double)

train_X = torch.rand(256, 2)
train_Y = torch.sin(train_X * 10).sum(dim=-1, keepdim=True)

model = SingleTaskGP(train_X=train_X, train_Y=train_Y)
acqf = qNoisyExpectedImprovement(model, train_X, cache_root=False)

# rand_tensor = torch.rand(1024, 128, 128, dtype=torch.double)

def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((16, 4, 2))).contiguous().requires_grad_(True)
    loss = acqf(X_tensor).sum()
#     diff = rand_tensor - X_tensor
#     loss = (diff @ diff).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

bounds = Bounds(
    lb=np.zeros(128),
    ub=np.ones(128),
    keep_feasible=True,
)
x0 = np.random.random(128)

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"maxiter": 20},
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)
None

         3699 function calls (3561 primitive calls) in 0.008 seconds

   Ordered by: cumulative time
   List reduced from 472 to 47 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.008    0.008 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_minimize.py:53(minimize)
        1    0.000    0.000    0.008    0.008 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:290(_minimize_lbfgsb)
        1    0.000    0.000    0.007    0.007 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:203(_prepare_scalar_function)
        1    0.000    0.000    0.007    0.007 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:166(__init__)
        2    0.000    0.000    0.007    0.004 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:71(_compute_if_need

In [4]:
res

  message: CONVERGENCE: NORM OF PROJECTED GRADIENT <= PGTOL
  success: True
   status: 0
      fun: 0.0
        x: [ 5.488e-01  7.152e-01 ...  1.919e-02  3.016e-01]
      nit: 0
      jac: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
     nfev: 1
     njev: 1
 hess_inv: <128x128 LbfgsInvHessProduct with dtype=float64>

In [2]:
%%time
def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((16, 4, 2))).contiguous().requires_grad_(True)
    loss = acqf._get_samples_and_objectives(X_tensor)[1].sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"ftol": 0, "gtol": 0, "maxiter": 20},
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)
None

         572464 function calls (546535 primitive calls) in 1.724 seconds

   Ordered by: cumulative time
   List reduced from 431 to 43 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.724    1.724 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_minimize.py:53(minimize)
        1    0.001    0.001    1.724    1.724 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:290(_minimize_lbfgsb)
      206    0.000    0.000    1.711    0.008 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:341(fun_and_grad)
      402    0.002    0.000    1.710    0.004 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:71(_compute_if_needed)
      207    0.000    0.000    1.709    0.008 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:29